In [1]:
# import module
import os
import re
import numpy as np
import pandas as pd
import cv2
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import argparse
import imutils
from tqdm import tqdm

In [2]:
# 파일 이름 불러오기
img_lst = []
lst_age = [20,30,40,50]
lst_sex = ['man','woman']
for age in lst_age:
    for sex in lst_sex:
        img_lst.append(glob(f'image/{sex}/{age}/*.JPG'))
        
img_lst_final = glob('images_final/*.jpg')

In [3]:
# SSIM Test
imageA = cv2.imread(img_lst[0][0])
imageB = cv2.imread(img_lst_final[1])
imageA = cv2.resize(imageA, (128,128))
imageB = cv2.resize(imageB, (128,128))
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
(score, diff) = ssim(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

print("SSIM : {}".format(score))

SSIM : 0.15698801147963456


In [4]:
# 빈 리스트 생성
lst_ssim = []

# 한 이미지에 대한 SSIM 점수 측정.
for lst in tqdm(img_lst):
    for i in lst:
        for img in img_lst_final:
            imageA = cv2.imread(lst[0])
            imageB = cv2.imread(img)
            imageC = cv2.resize(imageA, (128,128))
            imageD = cv2.resize(imageB, (128,128))
            grayC = cv2.cvtColor(imageC, cv2.COLOR_BGR2GRAY)
            grayD = cv2.cvtColor(imageD, cv2.COLOR_BGR2GRAY)
            (score, diff) = ssim(grayC, grayD, full=True)
            diff = (diff * 255).astype("uint8")
            lst_ssim.append([i[13:],img[13:], score, sex, age])

100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  4.61it/s]


In [5]:
# 한 이미지에 대한 SSIM 값 Top 5.
df_ssim = pd.DataFrame(lst_ssim, columns=['origin_jpg','images','score','sex','age'])

In [6]:
df_ssim = df_ssim.sort_values(by=['score'], axis=0,ascending=False); df_ssim

,origin_jpg,images,score,sex,age
65,0/2DT02_010243.JPG,-5_0.jpg,0.479232,woman,50
47,0/2DT02_008173.JPG,-5_0.jpg,0.460550,woman,50
66,0/2DT02_010243.JPG,-7_0.jpg,0.423493,woman,50
48,0/2DT02_008173.JPG,-7_0.jpg,0.412049,woman,50
29,0/2DT02_005151.JPG,-5_0.jpg,0.404434,woman,50
...,...,...,...,...,...
53,0/2DT02_008173.JPG,-9_0.jpg,0.101006,woman,50
62,2DT01_000448.JPG,-9_0.jpg,0.100123,woman,50
17,0/2DT01_006183.JPG,-9_0.jpg,0.099862,woman,50
71,0/2DT02_010243.JPG,-9_0.jpg,0.093699,woman,50


In [7]:
# 가중평균(weighted average)
weight = []
for score in df_ssim['score']:
    weight.append(score/df_ssim['score'].sum())
    
# 각 weighted 값 계산
weight

[0.028532904129451288,
 0.027420583059330372,
 0.0252142427657877,
 0.02453289751053048,
 0.02407949823643567,
 0.02401594051928801,
 0.02278749365128546,
 0.021953349815394478,
 0.021014295737513754,
 0.02073708160563255,
 0.02057230575930138,
 0.020146750907293315,
 0.01994454837239756,
 0.019619515152634347,
 0.019561296030631642,
 0.019512839714377218,
 0.01800567953607672,
 0.01770313206505925,
 0.017506499569813094,
 0.0174496164145856,
 0.01718799635284219,
 0.01713731184023081,
 0.01663798771509677,
 0.01640815516388536,
 0.016359906344461845,
 0.016177620642628423,
 0.014940927128413058,
 0.014674593913966824,
 0.014592468076167786,
 0.014544572151526405,
 0.014480864907475624,
 0.014465232113228838,
 0.014136881377808651,
 0.013613080851207991,
 0.013221023797681749,
 0.013140114653675455,
 0.012171371770142384,
 0.012011921232938325,
 0.011967329417923334,
 0.011750610808498103,
 0.011540043531807416,
 0.011539670212978841,
 0.011479372454579823,
 0.011382649598518878,
 0.01

In [8]:
# dim 값 불러오기
img_factor = pd.read_csv("tables/img_factors.csv")
item_factor = pd.read_csv("tables/item_factors.csv")

In [9]:
# 각 이미지에 대한 임베딩 값
img_top5_dim = []
for dim in df_ssim['images']:
    img_top5_dim.append(img_factor.query(f'target == "{dim}"').iloc[0])
df_img_ssim = pd.DataFrame(img_top5_dim);df_img_ssim

,target,crop,dim01,dim02,dim03,dim04,dim05,dim06,dim07,dim08,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
7066,-5_0.jpg,-5_0.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
7066,-5_0.jpg,-5_0.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
6803,-7_0.jpg,-7_1.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
6803,-7_0.jpg,-7_1.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
7066,-5_0.jpg,-5_0.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,-9_0.jpg,-9_2.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
3427,-9_0.jpg,-9_2.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
3427,-9_0.jpg,-9_2.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
3427,-9_0.jpg,-9_2.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365


In [10]:
# 임베딩값만 가져오기
img_emb_value = df_img_ssim.iloc[:,2:]; img_emb_value

,dim01,dim02,dim03,dim04,dim05,dim06,dim07,dim08,dim09,dim10,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
7066,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
7066,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
6803,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
6803,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
7066,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
3427,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
3427,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
3427,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365


In [11]:
# 산술평균(arithmetic mean)
img_arithmetic_mean = []
for dim in img_emb_value.columns:
    img_arithmetic_mean.append(img_emb_value[dim].sum()/5)

In [12]:
# 임베딩 값 추가
# img_imb_value.append(pd.DataFrame([img_arithmetic_mean], index=['arithmetic_mean'], columns=img_imb_value.columns))

In [13]:
# 한 이미지에 대한 가중평균 값.
img_w_avg = []
for dim,w in zip(range(len(img_emb_value.index)), weight):
    img_w_avg.append(img_emb_value.loc[img_emb_value.index[dim]]*w)
df_img_w_avg = sum(img_w_avg);df_img_w_avg


KeyboardInterrupt



In [14]:
item_top5_dim = []
for dim in df_ssim['images']:
    item_top5_dim.append(item_factor.query(f'target == "{dim}"').iloc[0])
df_ssim_item = pd.DataFrame(item_top5_dim);df_ssim_item

,target,dim01,dim02,dim03,dim04,dim05,dim06,dim07,dim08,dim09,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
741,-5_0.jpg,0.322444,0.066134,-0.386311,0.104063,-0.260146,0.208860,-0.127294,-0.260703,0.117243,...,0.033310,0.198505,-0.173609,0.349440,-0.055789,-0.054007,-0.007184,0.372787,-0.008929,-0.261084
741,-5_0.jpg,0.322444,0.066134,-0.386311,0.104063,-0.260146,0.208860,-0.127294,-0.260703,0.117243,...,0.033310,0.198505,-0.173609,0.349440,-0.055789,-0.054007,-0.007184,0.372787,-0.008929,-0.261084
766,-7_0.jpg,1.682376,-1.373034,1.029325,1.559954,1.356046,-1.283311,-1.437937,-1.673967,1.494577,...,1.378859,-1.235597,1.213370,-0.868012,-1.443011,1.455716,-1.521135,-1.072522,1.496875,-1.500100
766,-7_0.jpg,1.682376,-1.373034,1.029325,1.559954,1.356046,-1.283311,-1.437937,-1.673967,1.494577,...,1.378859,-1.235597,1.213370,-0.868012,-1.443011,1.455716,-1.521135,-1.072522,1.496875,-1.500100
741,-5_0.jpg,0.322444,0.066134,-0.386311,0.104063,-0.260146,0.208860,-0.127294,-0.260703,0.117243,...,0.033310,0.198505,-0.173609,0.349440,-0.055789,-0.054007,-0.007184,0.372787,-0.008929,-0.261084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,-9_0.jpg,1.257450,0.979704,1.225068,-1.303765,1.337282,1.483602,-0.972053,-1.236881,-1.188288,...,-1.197970,-1.218384,1.246854,1.451848,1.219470,1.138558,1.274744,1.256713,1.285686,-1.268280
773,-9_0.jpg,1.257450,0.979704,1.225068,-1.303765,1.337282,1.483602,-0.972053,-1.236881,-1.188288,...,-1.197970,-1.218384,1.246854,1.451848,1.219470,1.138558,1.274744,1.256713,1.285686,-1.268280
773,-9_0.jpg,1.257450,0.979704,1.225068,-1.303765,1.337282,1.483602,-0.972053,-1.236881,-1.188288,...,-1.197970,-1.218384,1.246854,1.451848,1.219470,1.138558,1.274744,1.256713,1.285686,-1.268280
773,-9_0.jpg,1.257450,0.979704,1.225068,-1.303765,1.337282,1.483602,-0.972053,-1.236881,-1.188288,...,-1.197970,-1.218384,1.246854,1.451848,1.219470,1.138558,1.274744,1.256713,1.285686,-1.268280


In [15]:
# 임베딩값만 가져오기
emb_value_item = df_ssim_item.iloc[:,1:]

# user_factor 산술평균(arithmetic mean)
arithmetic_mean_item = []
for dim in emb_value_item.columns:
    arithmetic_mean_item.append(emb_value_item[dim].sum()/5)

In [16]:
# 임베딩 값 추가
# imb_value_item.append(pd.DataFrame([arithmetic_mean_item], index=['arithmetic_mean'], columns=imb_value_item.columns))

In [ ]:
# 한 이미지에 대한 가중평균 값.
df_item_w_avg = []
for dim, w in zip(range(len(emb_value_item.index)), weight):
    df_item_w_avg.append(emb_value_item.loc[emb_value_item.index[dim]]*w)
df_item_w_average = sum(df_item_w_avg);df_item_w_average